
# EEG Data from EDF File


An Example of reading and working with EDF File.

To read EDF file, 'pyedflib' library is needed, which is not installed as requirements of spkit.
To install pyedflib use 'pip install pyedflib'


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, re, time, glob, requests
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


import spkit as sp
print('spkit-version : ',sp.__version__)

## Download sameple file 



In [ ]:
# Download file, if not already downloaded


path ='https://github.com/spkit/data_samples/raw/main/files/Resting_EPOCX_14Ch_Sample1.edf'


file_name = 'Resting_EPOCX_14Ch_Sample1.edf'

if not(os.path.exists(file_name)):
    req = requests.get(path)
    with open(file_name, 'wb') as f:
            f.write(req.content)

## Read EDF File 



In [ ]:
X,fs,ch_labels,info = sp.io.read_bdf(file_name)
print('X shape: ',X.shape)
print('sampling freq: ', fs)
print('==')
print('Channels')
sp.utils.pretty_print(ch_labels)

## Extract EEG Data and process



In [ ]:
# Extract EEG
XE = X[:,6:20]
eeg_ch_names = ch_labels[6:20]
print(eeg_ch_names)

# Filter with 0.5 Hz Highpass
Xf = sp.filter_X(XE.copy(),fs=fs,band=[0.5], btype='highpass', ftype='SOS')

# Select 20 seconds only
Xf1 = Xf[:int(20*fs)]
t = np.arange(Xf1.shape[0])/fs

# compute power

_,Pm,_ = sp.eeg.rhythmic_powers(X=Xf1.copy(),fs=fs,fBands=[[4],[8,14]],Sum=False,Mean=True,SD =False)
Pm = 20*np.log10(Pm)

## Plot - cover



In [ ]:
# this is for the cover
fig,ax = plt.subplots(1,2,figsize=(10,4),gridspec_kw={'width_ratios': [2, 1]})
sep=100
ax[0].plot(t,Xf1+np.arange(Xf1.shape[1])*sep,color='k',alpha=0.5)
ax[0].set_xlim([t[0],t[-1]])
ax[0].set_yticks(np.arange(Xf1.shape[1])*sep, eeg_ch_names)
ax[0].set_xlabel('time (s)')
ax[0].set_title('EEG Sample: 20s duration')
ax[0].text(17,sep*7,'EDF File',fontsize=130,ha='center',va='center',color='C3')
Z1,im1 = sp.eeg.topomap(data=Pm[1],ch_names=eeg_ch_names,shownames=False,axes=ax[1],return_im=True,cmap='Blues_r')
ax[1].set_title(r'$\alpha$ (8-14 Hz)')
ax[1].set_zorder(-1)
#plt.colorbar(im1, ax=ax[1],label='dB')
plt.show()

## Plot



In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,4),gridspec_kw={'width_ratios': [2, 1]})
sep=100
ax[0].plot(t,Xf1+np.arange(Xf1.shape[1])*sep,color='C0')
ax[0].set_xlim([t[0],t[-1]])
ax[0].set_yticks(np.arange(Xf1.shape[1])*sep, eeg_ch_names)
ax[0].set_xlabel('time (s)')
ax[0].set_title('EEG Sample: 20s duration')

Z1,im1 = sp.eeg.topomap(data=Pm[1],ch_names=eeg_ch_names,shownames=False,axes=ax[1],return_im=True)

ax[1].set_title(r'$\alpha$ (8-14 Hz)')
plt.colorbar(im1, ax=ax[1],label='dB')
plt.show()